## 특징점
> 특징점이란 '영상에서 특징이 될 만한 지점', '영상에서 중요 정보를 가지고 있다고 판단되는 지점'이다.

- 특징점을 통해서 각기 다른 이미지, 영상간의 대응 관계를 수립할 수 있다.
- 그렇다면, 특징점 Matching의 세부 과정을 살펴보자.

**특징점 Matching 세부과정**
1. Detection - 검출 : 독특한 특징점들을 검출함
2. Description - 기술 : 특징점 주위의 픽셀들이 어떤 모양을 가지는지 정량적인 벡터로 도출 = feature descripter
    - ex) 특징점을 기준으로 한 local window의 주변 픽셀들의 밝기 값을 비교했을 때 차이가 적으면 같음을 인식할 수 있다. 이렇게, 서로 다른 영상의 '특징점이 서로 유사한지 비교할 수 있는 데이터'가 descripter인 것이다! </br></br>
3. Matching - 매칭 : 특징 벡터간의 거리를 계산하여 대응 관계를 수립한다 -> 거리가 짧은 특징점 벡터들끼리 정합을 하면 특징점간의 대응 관계가 수립이 된다!
***

**좋은 특징점 및 특징기술자의 특성**
1. 구별가능성 Distinctiveness </br>
: 같거나 다름을 구별하기 위해서 독특한 특징을 지니고 있어야 한다. 그러한 구별가능성을 갖고 있는 점을 특징점으로 설정한다.

2. 반복가능성 Repeatability </br>
: 두 영상에서 동일한 지점이 어느 정도 이상 특징적으로 검출되게 하고자 한다.

3. 소형화 Compactness & 효율성 Efficiency</br> 
: 검출된 특징점을 중심으로 local window를 잡고 그 윈도우의 픽셀의 분포에 대한 벡터 표현 즉, 기술자를 생성할때 그 기술자 자체가 최대한 작은 것이 좋다.</br>
->why?) 특징점들을 기술자를 통해 정합을 할 때, 여러가지 경우의 수를 고려해야 하므로 계산의 효율성에 있어 기술자의 크기가 작을수록 좋다! + 데이터 사이즈 측면에서 저장에도 용이함

***

**특징점 검출과정**
- 도출된 특징점이 상대 영상의 어느 픽셀 위치와 대응되는지는 미리 알 수 없다.
- 그러나 특징점으로 뽑은 픽셀에서의 작은 위치 변화에 따라 생김새appearance가 얼마나 안정적인지 아닌지를 계산할 수 있음.
    - 특징점을 중심으로하는 local window의 픽셀의 변화가 크다 -> 중요한 정보로 생각할 수 있음.
- 영상의 패치를 주변 이웃과 비교함

**Corner Detection**
- 작은 윈도우 내부에서 모서리Corner를 인식함.
- 어느 방향으로 이동하든 픽셀 밝기값의 intensity분포가 크게 변화되는 window를 찾고자 함.

![2](img/2.png)

1. Flat region : 모든 방향으로 변화 없음
2. Edge : 가장자리 방향으로 변화 없음
3. Corner : 모든 방향으로 변화함 -> 특징점이 될 수 있다! </br>
-> why?) 어느 방향으로 움직이든 변화가 크다는 것은 그 점이 독특한 특성을 갖고 있다는 것이고, 이에 따라 그 점의 위치가 드러나기 때문이다. 그러므로 '구별가능성'이 생기는 것이다! </br>
-> 즉, Corner point가 keypoint = feature point가 될 수 있다! </br>

그렇다면, Corner점을 어떻게 찾을 수 있을까?

***

**Auto-correlation에 의한 코너 검출**
- 자기 자신과 주변 영역간의 상관관계를 계산하는 것을 의미한다

![3](img/3.png)

- 위의 그림과 수식을 보면, keypoint가 될 수 있는 특정한 점을 기준으로 한 window를 u, v만큼 옮기고 겹치는 부분의 픽셀간의 차의 제곱의 합을 구하는 것이다.
- w(x, y) : Window function / I(x+u, y+v) : Shifted intensity / I(x, y) : intensity
- 위의 값이 높을수록 변화가 크다고 생각하면 된다.
- 즉, 작은 변화에 대해 E(u, v)값이 어떻게 변화되는지 관찰하고자 하는 것이다. 그러나, 이 계산을 전부 수행하는 데에는 아주 많은 시간이 소요된다.
    - O(window_width^2 * shift_range^2 * image_width^2) </br>

![4](img/4.png)
![5](img/5.png)

- 위의 그림처럼 강한 봉우리를 갖는 지점을 찾고자 하는 것이다!
- 그러므로, E(u, v)의 좁은 지역을 국소적으로 2차 표면으로 근사하는 방법을 택한다! -> '테일러 급수 전개'활용

![6](img/6.png)

- (0, 0)근방에서 E(u, v)의 2계도 근사값은 2차 테일러 급수 전개에 의해 도출된다.
- 위의 수식에서 E(0, 0)은 f(0, 0)의 값, [u, v]는 (x - (0,0))을 의미, 2x1행렬은 각각 u, v에 대한 편미분값, 두번째 항에서 2x2행렬은 u, v에 대한 2계도 편미분값을 의미하고 [u, v]와 전치행렬의 곱은 xT*x를 의미한다. 즉, 2차 테일러 급수식에 해당하는 것이다!

![7](img/7.png)
![8](img/8.png)
![9](img/9.png)

- 이렇게 얻은 (0, 0)근방에서 E(u, v)의 2계도 근사식에서 우리는 얼마나 봉우리가 깊은지, 즉 2차함수가 얼마나 가파른지에 대한 정보를 얻고 싶은 것이다!
- 그렇다면, 2차함수의 가파름은 2계도 미분값에 의해 결정되므로, 앞의 항들은 0으로 설정하고 2계도 미분값만 확인하도록 한다.






![10](img/10.png)

- M은 결국 2계도함수를 이용하여 만들어진 Hessian행렬이다!

***Hessian Matrix***
- Hessian행렬은 어떤 함수의 2계 도함수들을 이용하여 행렬을 만든 것이다.
- 헤시안 행렬의 원소는 모두 이계도함수로 이루어져 있는데, f의 이계도함수가 연속이라면 헤세 행렬은 '대칭 행렬'이다.

- <2차 미분계수의 의미>
- 2차 함수 f(x) = (1/2a)*x^2 + bx + c 가 있다고 하자. 이 함수 f(x)의 이계도함수는 a인데, a가 양수라면 아래로 볼록, 음수라면 위로 볼록한 함수 형태를 갖는다.
- a의 크기가 클수록 이차함수의 폭이 좁아져 더 볼록한 형태를 갖는다는 특징 또한 알 수 있다!

***Hessian Matrix의 기하학적 의미***
- 모든 행렬은 선형변환으로 대체해 생각할 수 있다. 그리고, 선형 변환을 기하하적으로 생각하면 선형 변환이란 일종의 공간 변형이라고도 생각할 수 있다.
- 기하학적으로 헤시안 행렬이 시행하고 있는 선형 변환은 기본 bowl형태의 함수를 좀 더 볼록하거나 오목하게 만드는 변환이다!
- 그렇다면 헤시안 행렬이 보여주는 기하학적인 변환의 주요한 특징을 파악하기 위해서는 어떻게 해야될까? -> 선형 변환의 중심축을 찾고, 그 크기를 수치화해야한다!
- 그러므로 헤시안 행렬의 고유값과 고유벡터를 파악함으로써 가능할 것이다!!!

cf) 선형 대수 개념
- M은 정사각 대칭 행렬이다.
- 대칭 행렬의 고유벡터들은 모두 서로 직교한다.
- 정사각 행렬 M은 M = P^-1AP를 만족시키는 임의의 행렬 P가 존재하면 대각화 할 수 있다.
- 이때 A는 대각행렬이며, P는 기저변화를 시키는 행렬이다.(2D에서는 P가 회전행렬이 된다.)

![11](img/11.png)

- 위와 같이 M행렬을 대각화하여 고유값에 의해 타원 축들의 길이가 결정되는 것을 알 수 있다.
- 방향을 나타내는 R행렬은 우리에게 중요치 않다! -> 우리는 Cornerness만 따지면 되므로, 방향은 의미가 없고 그러므로 M행렬을 대각화하여 얻은 고유값에 의해 Cornerness를 판단하면 된다!
- 더 큰 고유값일수록 기울기가 가파르고, 시각화할 때 더 작은 타원으로 나타내진다.

![12](img/12.png)
![13](img/13.png)

- 위의 그림과 수식을 살펴보자.
- 두 고유값이 둘 다 클때만 C값이 양수를 갖게끔 Cornerness Score를 정의한 것이다.
- 수식을 살펴보면, 고유값의 곱은 M이 대칭행렬이므로 M의 행렬식으로 표현할 수 있고, 정사각 대칭 행렬의 trace는 고유값의 합이므로 'C = det(M) - α * trace(M)^2'로 표현할 수 있다!! 결과는 밑의 수식과 같다!

![14](img/14.png)





***
**Harris corner detector**
- 각 윈도우에 대한 M행렬을 계산하여 cornerness score = C를 도출한다!
    - cf) 픽셀 당 이미지 미분값에서 M을 찾을 수 있다. </br>
- Cornerness score C중에 큰 값으 찾아내기 위해 임계값을 정하고 임계값 미만의 값은 걸러냄(C > threshold)
- 주변 픽셀 대비 국소적으로 C값이 최대인 local maximum 픽셀들을 선별한다.